In [0]:
# Run the code in colab
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Authenticate and create the PyDrive client.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
one = drive.CreateFile({'id':'1rfqtUeQMwqTd6kmM6HWLfcqYl0lgFqwJ'}) 
one.GetContentFile('train1.csv')
two = drive.CreateFile({'id':'1h-PescmsyHGsSFp-GH2x3XwS3q612TRu'}) 
two.GetContentFile('test1.csv')  

In [0]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import gc
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn import preprocessing
from sklearn.decomposition import PCA
from keras.layers import Dense, Input
from keras.models import Model
from keras import optimizers, regularizers
import numpy as np
import matplotlib.pyplot as plt

In [0]:
train_X = pd.read_csv('train1.csv')
gc.collect()

In [0]:
# Convert the primary_use column to one-hot codes
def onehotConvertor(data):
  pu = data.primary_use
  pu1 = np.array(pu).reshape(len(pu), -1)
  enc = OneHotEncoder()
  enc.fit(pu1)
  targets = enc.transform(pu1).toarray()
  data['pu1'] = targets[:,0]
  data['pu2'] = targets[:,1]
  data['pu3'] = targets[:,2]
  data['pu4'] = targets[:,3]
  data['pu5'] = targets[:,4]
  data['pu6'] = targets[:,5]
  data['pu7'] = targets[:,6]
  data['pu8'] = targets[:,7]
  data['pu9'] = targets[:,8]
  data['pu10'] = targets[:,9]
  data['pu11'] = targets[:,10]
  data['pu12'] = targets[:,11]
  data['pu13'] = targets[:,12]
  data['pu14'] = targets[:,13]
  data['pu15'] = targets[:,14]
  data['pu16'] = targets[:,15]
  data = data.drop('primary_use', axis = 1)
  return data

In [0]:
train_X = onehotConvertor(train_X)
gc.collect()

In [0]:
train_X = train_X.drop('floor_count', axis = 1)
train_X = train_X.dropna(axis=0, how='any')
gc.collect()

In [0]:
# Normalize the data
def normalize(data):
  data['building_id'] = stats.zscore(data['building_id'])
  data['timestamp'] = stats.zscore(data['timestamp'])
  data['square_feet'] = stats.zscore(data['square_feet'])
  data['year_built'] = stats.zscore(data['year_built'])
  data['air_temperature'] = stats.zscore(data['air_temperature'])
  data['dew_temperature'] = stats.zscore(data['dew_temperature'])
  data['cloud_coverage'] = stats.zscore(data['cloud_coverage'])
  data['precip_depth_1_hr'] = stats.zscore(data['precip_depth_1_hr'])
  data['wind_direction'] = stats.zscore(data['wind_direction'])
  data['wind_speed'] = stats.zscore(data['wind_speed'])
  return data

In [0]:
train_X = normalize(train_X)
gc.collect()

In [0]:
train_y = train_X.pop('meter_reading')
gc.collect()

In [0]:
test_X = pd.read_csv('test1.csv')
gc.collect()

In [0]:
test_X = onehotConvertor(test_X)
gc.collect()

In [0]:
test_X = test_X.drop('row_id', axis = 1)
test_X = test_X.drop('floor_count', axis = 1)
gc.collect()

In [0]:
test_X = normalize(test_X)
gc.collect()

In [4]:
# Construct the model
inputs = Input(shape=(28,))
encoded1 = Dense(14, activation='relu')(inputs)
encoded2 = Dense(7, activation='relu')(encoded1)
encoded3 = Dense(3, activation='relu')(encoded2)
outputs = Dense(1)(encoded3)
SSAE_model = Model(inputs, outputs)

In [0]:
# Pretrain AE1 
ipt = Input(shape=(28,))
encoded = Dense(14, activation='relu')(ipt)
decoded = Dense(28)(encoded)
autoencoder = Model(ipt, decoded)
encoder = Model(ipt, encoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
autoencoder.fit([train_X], [train_X], epochs=10, batch_size=100)
code = encoder.predict([train_X])
SSAE_model.layers[1].set_weights(encoder.layers[-1].get_weights())

In [0]:
# Pretrain AE2
ipt = Input(shape=(14,))
encoded = Dense(7, activation='relu')(ipt)
decoded = Dense(14)(encoded)
autoencoder = Model(ipt, decoded)
encoder = Model(ipt, encoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
autoencoder.fit([code], [code], epochs=10, batch_size=100)
code = encoder.predict([code])
SSAE_model.layers[2].set_weights(encoder.layers[-1].get_weights())

In [0]:
# Pretrain AE3
ipt = Input(shape=(7,))
encoded = Dense(3, activation='relu')(ipt)
decoded = Dense(7)(encoded)
autoencoder = Model(ipt, decoded)
encoder = Model(ipt, encoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
autoencoder.fit([code], [code], epochs=10, batch_size=100)
code = encoder.predict([code])
SSAE_model.layers[3].set_weights(encoder.layers[-1].get_weights())

In [0]:
# Pretrain AE4
ipt = Input(shape=(3,))
encoded = Dense(1, activation='relu')(ipt)
decoded = Dense(3)(encoded)
autoencoder = Model(ipt, decoded)
encoder = Model(ipt, encoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
autoencoder.fit([code], [code], epochs=10, batch_size=100)
code = encoder.predict([code])
SSAE_model.layers[4].set_weights(encoder.layers[-1].get_weights())

In [0]:
# Train DNN
SSAE_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
SSAE_model.fit([train_X], [train_y], epochs=50, batch_size=100)

In [0]:
# Test
test_y = SSAE_model.predict([test_X])